#  Problem statement : 


- Build classifier that predicts the conversion-likelihood of a user


# Content

I have divided this assignment into five notebooks for clarity purpose. One notebook with so many lines of codes could be overwhelming for viewer.
In this notebook, we shall go through data basics and some data preprocessing techniques. 

- Loading csv files, merging useful information from iata dataset to main dataste
- Calculating geographic distance between origins and destinations
- Extracting new features out of original dataset
- Dealing with time series and time related variables & handing other data problems such as missing values
- Finally, saving data for next steps

# 1)-Importing key modules

In [1]:
# To avoid warnings Message 
import warnings
warnings.filterwarnings('ignore')
# For processing
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy
import datetime as dt
from datetime import date
import datetime as dt
# For visualization
import seaborn as sns
plt.rcParams["figure.figsize"] = (16, 10)
plt.rcParams["xtick.labelsize"] = 10
plt.figure(figsize=(16,10)) # this creates a figure 16 inch wide, 10 inch high
from pprint import pprint
%matplotlib inline

Populating the interactive namespace from numpy and matplotlib


# 2)-Loading data

In [2]:
df_event = pd.read_csv('events.csv')
df_event.shape

(47007, 9)

In [3]:
df_event.columns

Index(['ts', 'event_type', 'user_id', 'date_from', 'date_to', 'origin',
       'destination', 'num_adults', 'num_children'],
      dtype='object')

In [4]:
df_event.head()

,ts,event_type,user_id,date_from,date_to,origin,destination,num_adults,num_children
0,2017-04-27 11:06:51,search,60225f,2017-06-01,2017-06-07,PAR,NYC,6,1
1,2017-04-27 20:15:27,book,e5d69e,2017-08-12,2017-09-02,FRA,WAS,3,1
2,2017-04-27 23:03:43,book,f953f0,2017-10-08,2017-10-11,BER,CGN,2,0
3,2017-04-27 15:17:50,book,794d35,2017-04-28,2017-05-01,BER,BCN,1,0
4,2017-04-27 22:51:57,book,ca4f94,2017-05-16,2017-05-22,DEL,BKK,4,0


In [5]:
df_event.event_type.unique()

array(['search', 'book'], dtype=object)

So we have a binary classification problem with our goal to predict the conversion-likelihood of a user

In [6]:
df_geo = pd.read_csv('iata_1_1.csv')

df_geo.shape

(500, 3)

In [7]:
df_geo.head()

,iata_code,lat,lon
0,LON,51.50,-0.17
1,MOW,55.75,37.70
2,NYC,40.71,-74.01
3,PAR,48.85,2.35
4,IST,41.01,28.95


In [8]:
df_geo[df_geo.iata_code == "PAR"]

,iata_code,lat,lon
3,PAR,48.85,2.35


In [9]:
df_geo[df_geo.iata_code == "NYC"]

,iata_code,lat,lon
2,NYC,40.71,-74.01


# 3)-Combining dataset

### a)- Working with orgin 

In [10]:
df_origin=pd.DataFrame(columns=["origin"])
df_origin['origin']=df_event['origin']
df_origin['origin_lat']=np.nan
df_origin.head(2)

,origin,origin_lat
0,PAR,NaN
1,FRA,NaN


In [11]:
for k in range(len(df_origin)):
    df_origin['origin_lat'][k] = (df_geo['lat'][df_geo['iata_code']==df_origin['origin'][k]][df_geo['lat'][df_geo['iata_code']==df_origin['origin'][k]].index[0]])

In [12]:
df_origin.head(2)

,origin,origin_lat
0,PAR,48.85
1,FRA,50.12


In [13]:
df_origin['origin_lon']=np.nan

In [14]:
for k in range(len(df_origin)):
    df_origin['origin_lon'][k] = (df_geo['lon'][df_geo['iata_code']==df_origin['origin'][k]][df_geo['lon'][df_geo['iata_code']==df_origin['origin'][k]].index[0]])

In [15]:
df_origin.head(2)

,origin,origin_lat,origin_lon
0,PAR,48.85,2.35
1,FRA,50.12,8.68


In [16]:
df_origin.shape

(47007, 3)

### b)-working with destination 

In [17]:
df_destination=pd.DataFrame(columns=["destination"])

In [18]:
df_destination['destination']=df_event['destination']
df_destination.head(2)

,destination
0,NYC
1,WAS


In [19]:
df_destination['destination_lat']=np.nan
df_destination['destination_lon']=np.nan

In [20]:
for k in range(len(df_destination)):
    df_destination['destination_lat'][k] = (df_geo['lat'][df_geo['iata_code']==df_destination['destination'][k]][df_geo['lat'][df_geo['iata_code']==df_destination['destination'][k]].index[0]])
    df_destination['destination_lon'][k] = (df_geo['lon'][df_geo['iata_code']==df_destination['destination'][k]][df_geo['lon'][df_geo['iata_code']==df_destination['destination'][k]].index[0]])

In [21]:
df_destination.head(2)

,destination,destination_lat,destination_lon
0,NYC,40.71,-74.01
1,WAS,38.90,-77.04


**Adding to main dataset**

In [22]:
df_event['origin_lat'] = df_origin['origin_lat']
df_event['origin_lon'] = df_origin['origin_lon']

In [23]:
df_event['destination_lat']= df_destination['destination_lat']
df_event['destination_lon']=df_destination['destination_lon']

In [24]:
df_event.head(2)

,ts,event_type,user_id,date_from,date_to,origin,destination,num_adults,num_children,origin_lat,origin_lon,destination_lat,destination_lon
0,2017-04-27 11:06:51,search,60225f,2017-06-01,2017-06-07,PAR,NYC,6,1,48.85,2.35,40.71,-74.01
1,2017-04-27 20:15:27,book,e5d69e,2017-08-12,2017-09-02,FRA,WAS,3,1,50.12,8.68,38.90,-77.04


# 4)- Calculating distance

For geographic distance between origins and destinations, we choose Haversine distance between two points 

Reference for this : 

- https://www.movable-type.co.uk/scripts/latlong.html
- https://kanoki.org/2019/12/27/how-to-calculate-distance-in-python-and-pandas-using-scipy-spatial-and-distance-functions/

In [25]:
def haversine_vectorize(lon1, lat1, lon2, lat2):
 
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
 
    newlon = lon2 - lon1
    newlat = lat2 - lat1
 
    haver_formula = np.sin(newlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(newlon/2.0)**2
 
    dist = 2 * np.arcsin(np.sqrt(haver_formula ))
    km = 6367 * dist #6367 for distance in KM for miles use 3958
    return km

In [26]:
haversine_vectorize(df_event['origin_lon'],df_event['origin_lat'],df_event['destination_lon'],
                   df_event['destination_lat'])

0        5834.154716
1        6525.926149
2         469.781624
3        1498.817537
4        2921.339028
            ...     
47002     856.002808
47003     395.323222
47004     730.610705
47005     174.707494
47006     614.303469
Length: 47007, dtype: float64

In [27]:
df_event['distance'] = haversine_vectorize(df_event['origin_lon'],df_event['origin_lat'],df_event['destination_lon'],
                   df_event['destination_lat'])

In [28]:
df_event.head(2)

,ts,event_type,user_id,date_from,date_to,origin,destination,num_adults,num_children,origin_lat,origin_lon,destination_lat,destination_lon,distance
0,2017-04-27 11:06:51,search,60225f,2017-06-01,2017-06-07,PAR,NYC,6,1,48.85,2.35,40.71,-74.01,5834.154716
1,2017-04-27 20:15:27,book,e5d69e,2017-08-12,2017-09-02,FRA,WAS,3,1,50.12,8.68,38.90,-77.04,6525.926149


**Distance will be one of our key extracted feature for analysis**

# 5)- Creating new feature

num_family

In [29]:
df_event["num_family"]=df_event['num_adults']+df_event['num_children']

In [30]:
df_event.head(2)

,ts,event_type,user_id,date_from,date_to,origin,destination,num_adults,num_children,origin_lat,origin_lon,destination_lat,destination_lon,distance,num_family
0,2017-04-27 11:06:51,search,60225f,2017-06-01,2017-06-07,PAR,NYC,6,1,48.85,2.35,40.71,-74.01,5834.154716,7
1,2017-04-27 20:15:27,book,e5d69e,2017-08-12,2017-09-02,FRA,WAS,3,1,50.12,8.68,38.90,-77.04,6525.926149,4


drop extra columns for clarity. Also if we keep then they will cause multi-collinearity. W´shall us´extracted featurs for our analysis

In [31]:
df_event.drop(['num_adults', 'num_children','origin_lat','origin_lon','destination_lat','destination_lon'], axis=1, inplace=True)

In [32]:
df_event.head(2)

,ts,event_type,user_id,date_from,date_to,origin,destination,distance,num_family
0,2017-04-27 11:06:51,search,60225f,2017-06-01,2017-06-07,PAR,NYC,5834.154716,7
1,2017-04-27 20:15:27,book,e5d69e,2017-08-12,2017-09-02,FRA,WAS,6525.926149,4


# 6)-Working with Time Series

Let's suppose. we want to see which year we have least/maximum online travel activities i.e search or booking. 

In [33]:
# Checking all variables with their data-type
def _tbl_dtype(dataset):
    sum_dtype = pd.DataFrame(dataset.dtypes).sort_values(0).rename(columns = {0:'Data Type'})
    return sum_dtype

table_dtype = _tbl_dtype(df_event)
table_dtype

,Data Type
num_family,int64
distance,float64
ts,object
event_type,object
user_id,object
date_from,object
date_to,object
origin,object
destination,object


### 6.1)-convert object to datetime

In [34]:
df_event["ts_datetime"]=pd.to_datetime(df_event.ts)
df_event["date_from_datetime"]=pd.to_datetime(df_event.date_from)
df_event["date_to_datetime"]=pd.to_datetime(df_event.date_to)

In [35]:
table_dtype = _tbl_dtype(df_event)
table_dtype

,Data Type
num_family,int64
distance,float64
ts_datetime,datetime64[ns]
date_from_datetime,datetime64[ns]
date_to_datetime,datetime64[ns]
ts,object
event_type,object
user_id,object
date_from,object
date_to,object


In [36]:
df_event.head(2)

,ts,event_type,user_id,date_from,date_to,origin,destination,distance,num_family,ts_datetime,date_from_datetime,date_to_datetime
0,2017-04-27 11:06:51,search,60225f,2017-06-01,2017-06-07,PAR,NYC,5834.154716,7,2017-04-27 11:06:51,2017-06-01,2017-06-07
1,2017-04-27 20:15:27,book,e5d69e,2017-08-12,2017-09-02,FRA,WAS,6525.926149,4,2017-04-27 20:15:27,2017-08-12,2017-09-02


**drop extra columns**

In [37]:
df_event.drop(['ts', 'date_to','date_from'], axis=1, inplace=True)

In [38]:
df_event.head(2)

,event_type,user_id,origin,destination,distance,num_family,ts_datetime,date_from_datetime,date_to_datetime
0,search,60225f,PAR,NYC,5834.154716,7,2017-04-27 11:06:51,2017-06-01,2017-06-07
1,book,e5d69e,FRA,WAS,6525.926149,4,2017-04-27 20:15:27,2017-08-12,2017-09-02


### 6.2)- Handling missing values

In [39]:
df_event.isnull().sum()

event_type             0
user_id                0
origin                 0
destination            0
distance               0
num_family             0
ts_datetime            0
date_from_datetime    22
date_to_datetime       3
dtype: int64

We need to solve this missing value issue before anything

In [40]:
# to check what ate exact data samples that contain missing values
df_event[df_event["date_from_datetime"].isnull()]

,event_type,user_id,origin,destination,distance,num_family,ts_datetime,date_from_datetime,date_to_datetime
5476,search,46cad7,AMS,MUC,670.157951,0,2017-04-28 23:24:26,NaT,2017-05-24
7580,search,46cad7,AMS,MUC,670.157951,0,2017-04-28 23:41:06,NaT,2017-05-24
7687,book,46cad7,AMS,MUC,670.157951,0,2017-04-28 23:52:17,NaT,2017-05-24
8407,search,46cad7,AMS,MUC,670.157951,0,2017-04-28 23:25:25,NaT,2017-05-24
13796,search,299320,BRU,TLS,834.874939,0,2017-04-23 19:02:48,NaT,NaT
15677,search,f78263,DUS,CPT,9535.990040,0,2017-04-29 16:16:49,NaT,2018-02-23
18675,search,2ac874,AYT,DUS,2472.273023,0,2017-04-24 21:13:32,NaT,NaT
19137,search,139d74,CGN,AYT,2432.590282,0,2017-04-29 21:31:57,NaT,2017-09-02
20664,search,450011,DUS,BKK,9069.518965,0,2017-04-27 19:35:16,NaT,2017-07-10
23407,search,450011,BKK,DUS,9069.518965,0,2017-04-27 17:52:49,NaT,2018-04-02


these are our points where we get missing values.

**Solution**

Some like to remove missing values. I am not one of them. I like to keep my data and this data has already get very close to my heart. So, I ll keep data.

For keeping data, I can use impute or fill method. Impute is mostly used if missing data is in numeric variable column then we use "mean" or "median" to fill value.

I will use ffill i.e forward fill to deal with data. There are many other ways but, it is simple and we do not have many missing values in our data. We have only 22 considering out of 47007......is a small number. 


In [41]:
df_event=df_event.fillna(method="ffill")

In [42]:
df_event.isnull().sum()

event_type            0
user_id               0
origin                0
destination           0
distance              0
num_family            0
ts_datetime           0
date_from_datetime    0
date_to_datetime      0
dtype: int64

### 6.3)-Adding extra Time Feature

- **journey_length (in Days)**

In [43]:
df_event['date_from_year']=df_event.date_from_datetime.dt.year
df_event['date_from_month']=df_event.date_from_datetime.dt.month
df_event['date_from_day']=df_event.date_from_datetime.dt.day
df_event['date_from_dayofweek']=df_event.date_from_datetime.dt.dayofweek

In [44]:
df_event.head()

,event_type,user_id,origin,destination,distance,num_family,ts_datetime,date_from_datetime,date_to_datetime,date_from_year,date_from_month,date_from_day,date_from_dayofweek
0,search,60225f,PAR,NYC,5834.154716,7,2017-04-27 11:06:51,2017-06-01,2017-06-07,2017,6,1,3
1,book,e5d69e,FRA,WAS,6525.926149,4,2017-04-27 20:15:27,2017-08-12,2017-09-02,2017,8,12,5
2,book,f953f0,BER,CGN,469.781624,2,2017-04-27 23:03:43,2017-10-08,2017-10-11,2017,10,8,6
3,book,794d35,BER,BCN,1498.817537,1,2017-04-27 15:17:50,2017-04-28,2017-05-01,2017,4,28,4
4,book,ca4f94,DEL,BKK,2921.339028,4,2017-04-27 22:51:57,2017-05-16,2017-05-22,2017,5,16,1


In [45]:
df_event.date_from_year.unique()

array([2017, 2018])

In [46]:
df_event.date_from_month.unique()

array([ 6,  8, 10,  4,  5,  1,  7, 11, 12,  2,  3,  9])

In [47]:
df_event['date_to_year']=df_event.date_to_datetime.dt.year
df_event['date_to_month']=df_event.date_to_datetime.dt.month
df_event['date_to_day']=df_event.date_to_datetime.dt.day
df_event['date_to_dayofweek']=df_event.date_to_datetime.dt.dayofweek

In [48]:
df_event.date_to_year.unique()

array([2017, 2018])

In [49]:
df_event.head()

,event_type,user_id,origin,destination,distance,num_family,ts_datetime,date_from_datetime,date_to_datetime,date_from_year,date_from_month,date_from_day,date_from_dayofweek,date_to_year,date_to_month,date_to_day,date_to_dayofweek
0,search,60225f,PAR,NYC,5834.154716,7,2017-04-27 11:06:51,2017-06-01,2017-06-07,2017,6,1,3,2017,6,7,2
1,book,e5d69e,FRA,WAS,6525.926149,4,2017-04-27 20:15:27,2017-08-12,2017-09-02,2017,8,12,5,2017,9,2,5
2,book,f953f0,BER,CGN,469.781624,2,2017-04-27 23:03:43,2017-10-08,2017-10-11,2017,10,8,6,2017,10,11,2
3,book,794d35,BER,BCN,1498.817537,1,2017-04-27 15:17:50,2017-04-28,2017-05-01,2017,4,28,4,2017,5,1,0
4,book,ca4f94,DEL,BKK,2921.339028,4,2017-04-27 22:51:57,2017-05-16,2017-05-22,2017,5,16,1,2017,5,22,0


In [50]:
df_event.date_to_day.unique()

array([ 7,  2, 11,  1, 22, 26, 14, 23,  6, 16,  3, 15, 13, 30,  4,  9, 29,
        5, 12, 18, 20, 17,  8, 19, 10, 28, 24, 27, 21, 25, 31])

In [51]:
df_event['len_jour'] = df_event['date_to_datetime'].sub(df_event['date_from_datetime'], axis=0)

In [52]:
df_event[["date_from_datetime","date_to_datetime","len_jour"]].head()

,date_from_datetime,date_to_datetime,len_jour
0,2017-06-01,2017-06-07,6 days
1,2017-08-12,2017-09-02,21 days
2,2017-10-08,2017-10-11,3 days
3,2017-04-28,2017-05-01,3 days
4,2017-05-16,2017-05-22,6 days


Now we get how long is expcted journey of our customer. But , it is in days. For analysis, we need numeric form 

In [53]:
df_event['len_jour'] = df_event['len_jour'] / np.timedelta64(1, 'D')

In [54]:
df_event[["date_from_datetime","date_to_datetime","len_jour"]].head()

,date_from_datetime,date_to_datetime,len_jour
0,2017-06-01,2017-06-07,6.0
1,2017-08-12,2017-09-02,21.0
2,2017-10-08,2017-10-11,3.0
3,2017-04-28,2017-05-01,3.0
4,2017-05-16,2017-05-22,6.0


In [55]:
df_event[["date_from_datetime","date_to_datetime","len_jour"]].tail()

,date_from_datetime,date_to_datetime,len_jour
47002,2017-04-30,2017-04-30,0.0
47003,2017-05-02,2017-05-04,2.0
47004,2017-05-16,2017-05-21,5.0
47005,2017-05-04,2017-05-04,0.0
47006,2017-05-04,2017-05-05,1.0


This information is interesting as it may describe our customer travelling habit. We may use this also as key extracted variable for our analysis

In [56]:
df_event.head()

,event_type,user_id,origin,destination,distance,num_family,ts_datetime,date_from_datetime,date_to_datetime,date_from_year,date_from_month,date_from_day,date_from_dayofweek,date_to_year,date_to_month,date_to_day,date_to_dayofweek,len_jour
0,search,60225f,PAR,NYC,5834.154716,7,2017-04-27 11:06:51,2017-06-01,2017-06-07,2017,6,1,3,2017,6,7,2,6.0
1,book,e5d69e,FRA,WAS,6525.926149,4,2017-04-27 20:15:27,2017-08-12,2017-09-02,2017,8,12,5,2017,9,2,5,21.0
2,book,f953f0,BER,CGN,469.781624,2,2017-04-27 23:03:43,2017-10-08,2017-10-11,2017,10,8,6,2017,10,11,2,3.0
3,book,794d35,BER,BCN,1498.817537,1,2017-04-27 15:17:50,2017-04-28,2017-05-01,2017,4,28,4,2017,5,1,0,3.0
4,book,ca4f94,DEL,BKK,2921.339028,4,2017-04-27 22:51:57,2017-05-16,2017-05-22,2017,5,16,1,2017,5,22,0,6.0


**drop extra columns**

In [57]:
df_event.drop(['date_from_datetime', 'date_to_datetime',"date_from_year","date_from_month","date_from_day","date_from_dayofweek"], axis=1, inplace=True)
df_event.drop(["date_to_year","date_to_month","date_to_day","date_to_dayofweek"], axis=1, inplace=True)

In [58]:
df_event.head(2)

,event_type,user_id,origin,destination,distance,num_family,ts_datetime,len_jour
0,search,60225f,PAR,NYC,5834.154716,7,2017-04-27 11:06:51,6.0
1,book,e5d69e,FRA,WAS,6525.926149,4,2017-04-27 20:15:27,21.0


**For our analysis, we shall use three key features**

- num_family (in number)
- len_jour (in days)
- distance (in km)

target variable will be same ofcourse.

# 7)- Saving data

In [59]:
df_event.to_csv('travel_data.csv',index=False)

**END OF NOTEBOOK1**